### Signate Competition using XGBRegressor
#### Predicting the number of audience form the given data

### Below will be setting for using this in Google collaboratory
#### Save all the files to ./signate folder

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir("drive/My Drive/signate")

In [ ]:
!ls

取り敢えず、与えられているcsvファイルをDataFrameとして読み込む。

In [1]:
import pandas as pd

def nan_checker(df):
    if sum(df.isnull()) > 0 :
        return False
    else:
        return True

df_holiday = pd.read_csv("holidays_in_japan.csv")
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_mapping = pd.read_csv("stadium_capacity_mapping.csv")
df_match = pd.read_csv("match_reports.csv")
df_sample = pd.read_csv("sample_submit.csv")

各DataFrameの先頭５行を確認

In [ ]:
df_holiday.head()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_mapping.head()

In [ ]:
df_match.head()

In [ ]:
df_sample.head()

## スタジアム
##### 文字列で記載されていて鬱陶しいので、人数に置き換える。

In [2]:
stadium_dic = df_mapping.to_dict()
av = sum(stadium_dic["capacity"].values())/len(stadium_dic["capacity"])
stadium_dic = df_mapping.to_dict(orient = "list")
li = []
new_di = {}
for key,value in zip(stadium_dic["stadium"],stadium_dic["capacity"]):
    new_di[key] = value
for i in df_train["venue"]:
    li.append(new_di[i])
df_train.venue = li

li = []
for i in df_test["venue"]:
    li.append(new_di[i])
df_test.venue = li

### 日付
#### 利用方法を検討中
#### 現在、祝日であればflagを立てるのみ

In [3]:
holiday_dic = df_holiday.to_dict(orient = "list")
holiday_dic = holiday_dic["holiday_date"]
li = []
for i in df_train["match_date"]:
    if i in holiday_dic:
        li.append(1)
    else:
        li.append(0)
df_train["holiday_flag"] = li
li = []
for i in df_test["match_date"]:
    if i in holiday_dic:
        li.append(1)
    else:
        li.append(0)
df_test["holiday_flag"] = li

In [4]:
df_train["match_date"] = [int(i.split("-")[0]) + int(i.split("-")[1])/13 for i in df_train["match_date"]]
df_test["match_date"] = [int(i.split("-")[0]) + int(i.split("-")[1])/13 for i in df_test["match_date"]]

### 時刻
#### 時刻の分布を確認し、時間毎に分ける

In [5]:
from collections import Counter

time = [int(i.split(":")[0]) for i in df_train["kick_off_time"]]
df_train["kick_off_time"] = time
time = [int(i.split(":")[0]) for i in df_test["kick_off_time"]]
df_test["kick_off_time"] = time

### 天気
#### 晴れ＞曇り＞雨なのでラベリングを地震で定義する。

In [6]:
#屋内＞晴＞曇＞霧＞雨＞雪＞雷
weather_di = {"屋":7,"晴":6,"曇":5,"霧":4,"雨":3,"雪":2,"雷":1}
df_train["weather"] = df_train["weather"].fillna(value = "晴")
df_test["weather"] = df_test["weather"].fillna(value = "晴")
df_train["weather"] = [weather_di[i[0]] for i in df_train["weather"]]
df_test["weather"] = [weather_di[i[0]] for i in df_test["weather"]]

### 気温と湿度

In [7]:
df_train["temperature"] = df_train["temperature"].fillna(value = df_train["temperature"].mean())
df_train["humidity"] = df_train["humidity"].fillna(value = df_train["humidity"].mean())
df_test["temperature"] = df_test["temperature"].fillna(value = df_test["temperature"].mean())
df_test["humidity"] = df_test["humidity"].fillna(value = df_test["humidity"].mean())

### Section and Round
#### 文字列を数値列に変換するだけ

In [8]:
#Section and round as integer

df_train["section"] = [int(i.lstrip("第").rstrip("節")) for i in df_train["section"]]
df_test["section"] = [int(i.lstrip("第").rstrip("節")) for i in df_test["section"]]
df_train["round"] = [int(i.lstrip("第").rstrip("日")) for i in df_train["round"]]
df_test["round"] = [int(i.lstrip("第").rstrip("日")) for i in df_test["round"]]

### Home team and Away team
#### ラベルエンコーディングを行うのみ

In [9]:
#dirty data : Transform hankaku -> zenkaku
import jaconv
df_train["home_team"] = [jaconv.h2z(i) for i in df_train["home_team"]]
df_train["away_team"] = [jaconv.h2z(i) for i in df_train["away_team"]]
df_test["home_team"] = [jaconv.h2z(i) for i in df_test["home_team"]]
df_test["away_team"] = [jaconv.h2z(i) for i in df_test["away_team"]]

In [10]:
temp = list(df_train["home_team"].to_dict().values())
temp2 = list(df_train["away_team"].to_dict().values())
temp3 = list(df_test["home_team"].to_dict().values())
temp4 = list(df_test["away_team"].to_dict().values())
temp.extend(temp2)
temp.extend(temp3)
temp.extend(temp4)
temp = set(temp)
counter = 0
team_di = {}
for i in temp:
    if i in team_di.values():
        pass
    else:
        team_di[i] = counter
        counter+=1
df_test["away_team"] = [team_di[i] for i in df_test["away_team"]]
df_test["home_team"] = [team_di[i] for i in df_test["home_team"]]
df_train["away_team"] = [team_di[i] for i in df_train["away_team"]]
df_train["home_team"] = [team_di[i] for i in df_train["home_team"]]

### 試合情報
#### 利用方法検討中


In [11]:
"""
import re
df_match["home_team_player1"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player1"]]
df_match["home_team_player2"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player2"]]
df_match["home_team_player3"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player3"]]
df_match["home_team_player4"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player4"]]
df_match["home_team_player5"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player5"]]
df_match["home_team_player6"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player6"]]
df_match["home_team_player7"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player7"]]
df_match["home_team_player8"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player8"]]
df_match["home_team_player9"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player9"]]
df_match["home_team_player10"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player10"]]
df_match["home_team_player11"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["home_team_player11"]]
df_match["away_team_player1"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player1"]]
df_match["away_team_player2"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player2"]]
df_match["away_team_player3"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player3"]]
df_match["away_team_player4"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player4"]]
df_match["away_team_player5"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player5"]]
df_match["away_team_player6"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player6"]]
df_match["away_team_player7"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player7"]]
df_match["away_team_player8"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player8"]]
df_match["away_team_player9"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player9"]]
df_match["away_team_player10"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player10"]]
df_match["away_team_player11"] = [re.sub("[0-9]+| |[A-Z]","",i) for i in df_match["away_team_player11"]]
"""
mix = df_train.append(df_test)
print(df_train.shape)
print(df_test.shape)
mix["score"] = df_match["away_team_score"] - -df_match["home_team_score"]
mix["away_score"] = df_match["away_team_score"]
mix["home_score"] = df_match["home_team_score"]
df_train = mix[:df_train.shape[0]]
df_test = mix[df_train.shape[0]:]
print(df_train.shape)
print(df_test.shape)



(3366, 14)
(477, 13)
(3366, 17)
(477, 17)


/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


### BroadCaster

In [12]:
def func(series):
    li = []
    for i in series:
        if "DAZN" in i:
            li.append(1)
        elif "J SPORTS" in i:
            li.append(2)
        elif "e2" in i:
            li.append(3)
        elif "プレミアムサービス" in i:
            li.append(4)
        else:
            li.append(5)
    return li

In [13]:
df_train["broadcasters"] = func(df_train["broadcasters"])
df_test["broadcasters"] = func(df_test["broadcasters"])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Attendanceが今回予想したい数値。
各説明変数との相関を確認する。

In [ ]:
df_test

### XGBoost
#### TODO: Parameter tuning

In [14]:
#DROP ID FOR BOTH TRAINGIN AND TEST

Y_train = df_train["attendance"]
X_train = df_train.drop(columns = ["attendance","id"])
X_test = df_test.drop(columns= ["attendance","id"])

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer
mod = XGBRegressor()

In [ ]:
params = {"learning_rate":[0.1,0.3,0.5],
        "max_depth": [2,3,5,10],
         "subsample":[0.5,0.8,0.9,1],
         "colsample_bytree": [0.5,1.0],
         }
# モデルにインスタンス生成
model = XGBRegressor()
# ハイパーパラメータ探索
cv = GridSearchCV(mod, params, cv = 10, scoring= 'neg_mean_squared_error', n_jobs =-1)

In [ ]:
cv.fit(X_train,Y_train)

In [ ]:
Y_test = cv.predict(X_test)

In [ ]:
#Remember that the first id 19075 is 39284 
X_test = pd.read_csv("./test.csv")
submission = pd.DataFrame({"id":X_test["id"],"attendance":Y_test})
submission.to_csv("./outXGBoost.csv",index=False)

### PyTorchを用いたLinearRegressionモデル

In [15]:
import torch
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F

In [16]:
X_test = Variable(torch.tensor(X_test.values)).float()
X_train = Variable(torch.tensor(X_train.values)).float()
Y_train = Variable(torch.tensor(Y_train.values)).float()

torch.Size([3366, 15])
torch.Size([477, 15])


In [17]:
class Linear(nn.Module):
    def __init__(self,i_dim):
        super(Linear,self).__init__()
        self.linear1 = nn.Linear(i_dim,200)
        self.linear2 = nn.Linear(200,100)
        self.linear3 = nn.Linear(100,50)
        self.linear4 = nn.Linear(50,10)
        self.linear5 = nn.Linear(10,1)
    def forward(self,i):
        i = self.linear1(i)
        i = self.linear2(i)
        i = self.linear3(i)
        i = self.linear4(i)
        i = self.linear5(i)
        return i
model = Linear(X_train.shape[1])

In [18]:
epochs = 10000
lr = 0.001

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr =lr)

In [19]:
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out.squeeze(),Y_train)
    loss.backward()
    optimizer.step()
    if(epoch % 100 == 0):
        print("epoch : " + str(epoch)+ " " + str(loss.item()))  

epoch : 0 464528256.0
epoch : 100 49927300.0
epoch : 200 46820316.0
epoch : 300 46072332.0
epoch : 400 46039520.0
epoch : 500 46005780.0
epoch : 600 45969380.0
epoch : 700 45927852.0
epoch : 800 45877720.0
epoch : 900 45810904.0
epoch : 1000 45707232.0
epoch : 1100 45552552.0
epoch : 1200 45733856.0
epoch : 1300 46263088.0
epoch : 1400 45098852.0
epoch : 1500 45056812.0
epoch : 1600 44979000.0
epoch : 1700 45009900.0
epoch : 1800 44960856.0
epoch : 1900 44934424.0
epoch : 2000 44922484.0
epoch : 2100 44915964.0
epoch : 2200 44935496.0
epoch : 2300 44911824.0
epoch : 2400 44905944.0
epoch : 2500 44902552.0
epoch : 2600 44941644.0
epoch : 2700 44899988.0
epoch : 2800 44896208.0
epoch : 2900 44893692.0
epoch : 3000 44902184.0
epoch : 3100 44891232.0
epoch : 3200 44888652.0
epoch : 3300 46715340.0
epoch : 3400 44888124.0
epoch : 3500 44884240.0
epoch : 3600 44882148.0
epoch : 3700 44909304.0
epoch : 3800 44880380.0
epoch : 3900 44877912.0
epoch : 4000 44879696.0
epoch : 4100 44877384.0
epo

In [20]:
prediction = model(X_test)
prediction = [i[0] for i in prediction.detach().numpy()]

torch.Size([477, 1])


In [23]:
X_test = pd.read_csv("test.csv")
submission = pd.DataFrame({"id":X_test["id"],"id2":prediction})
submission.to_csv("./outTorch.csv",index=False)